Lets make this agent like have it fill out all the information with single prompt

In [6]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [15]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

from langchain_core.tools import tool
from typing import List
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser

In [8]:
inp_simple = "Joe spent $5 and Hannah spent $8 on a toilet plunger"
inp_complex = """Sarah bought groceries for $60 and a bottle of wine for $30.
Jake paid for gas, which costs $50, and bought snacks for $25.
Emily booked $150 hotel room for her and Sarah and paid for dinner, which was $90.
Jake booked his own room for $80, and also bought a hotdog for Emily for $15.
Jake and Sarah also split a $20 popcorn which they shared with Emily"""

1. Extract the names of items, buyers, and consumers
2. Fill out information about how much value each contributor applied and consumer used
3. Pass to function that can resolve the debts

In [9]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0125")

In [10]:
class Purchase(BaseModel):
    """Information about a purchase"""
    name: str = Field(description="Descriptive name of the purchase")
    contributor_names: List[str] = Field(description="List of the names of people who contributed towards the purchase")
    consumer_names: List[str] = Field(description="List of the names of people who used/consumed the purchase")
        
        
# When I add a quote to purchases, it tends to attribute single item to single person

class Purchases(BaseModel):
    purchases: List[Purchase] = Field(description="List of purchases")

In [11]:
extract_names_prompt = ChatPromptTemplate.from_template(
"""
Extract the names of all people mentioned in the following passage.

Passage:
{input}

Return a comma-separated list of names.
"""
)


extract_names_chain = extract_names_prompt | llm

In [12]:
people = extract_names_chain.invoke(inp_complex)
people

AIMessage(content='Sarah, Jake, Emily', response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 119, 'total_tokens': 124}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-fb064825-5a81-45a1-b1db-c36c7322d3df-0')

In [19]:
extract_purchase_info_prompt = ChatPromptTemplate.from_template(
"""
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Purchases' function.

Derive the consumer of purchases by context. If not specified, list {people}.

Passage:
{input}
"""
)

extract_purchase_info_chain = extract_purchase_info_prompt | llm.with_structured_output(Purchases)

In [20]:
purchases_simple = extract_purchase_info_chain.invoke({"input": inp_simple, "people": "Joe, Hannah"})
purchases_simple

Purchases(purchases=[Purchase(name='toilet plunger', contributor_names=['Joe', 'Hannah'], consumer_names=['Joe', 'Hannah'])])

In [21]:
purchases_complex = extract_purchase_info_chain.invoke({"input": inp_complex, "people": "Sarah, Jake, Emily"})
purchases_complex

Purchases(purchases=[Purchase(name='groceries', contributor_names=['Sarah'], consumer_names=['Sarah', 'Jake', 'Emily']), Purchase(name='bottle of wine', contributor_names=['Sarah'], consumer_names=['Sarah', 'Jake', 'Emily']), Purchase(name='gas', contributor_names=['Jake'], consumer_names=['Sarah', 'Jake', 'Emily']), Purchase(name='snacks', contributor_names=['Jake'], consumer_names=['Sarah', 'Jake', 'Emily']), Purchase(name='hotel room', contributor_names=['Emily'], consumer_names=['Emily', 'Sarah']), Purchase(name='dinner', contributor_names=['Emily'], consumer_names=['Emily', 'Sarah']), Purchase(name='room booking', contributor_names=['Jake'], consumer_names=['Jake']), Purchase(name='hotdog', contributor_names=['Jake'], consumer_names=['Jake', 'Emily']), Purchase(name='popcorn', contributor_names=['Jake', 'Sarah'], consumer_names=['Sarah', 'Jake', 'Emily'])])

### Fill in information about the cost of items

Let's prep the response so we can pass it into the next prompt

In [22]:
def prep_purchases(purchases: Purchases):
    prepped_strings = []
    for i, purchase in enumerate(purchases.purchases, start=1):
        contributor_name_rows = "\n".join([
            f"| {name} | FILL IN |" for name in purchase.contributor_names
        ])

        purchase_string = "\n".join([
            f"{i}. {purchase.name} purchase",
            f"| Contributor | Individual contribution amount ($) |",
            "| ----------- | ---------------- |",
            contributor_name_rows,
            "| ----------- | ---------------- |",
            "| Sum | FILL IN |"
        ])
        prepped_strings.append(purchase_string)
    
    return prepped_strings

In [23]:
purchases_simple_string = "\n\n".join(prep_purchases(purchases_simple))
print(purchases_simple_string)

1. toilet plunger purchase
| Contributor | Individual contribution amount ($) |
| ----------- | ---------------- |
| Joe | FILL IN |
| Hannah | FILL IN |
| ----------- | ---------------- |
| Sum | FILL IN |


In [265]:
purchases_complex_string = "\n\n".join(prep_purchases(purchases_complex))
print(purchases_complex_string)

1. groceries purchase
| Contributor | Individual contribution amount ($) |
| ----------- | ---------------- |
| Sarah | FILL IN |
| ----------- | ---------------- |
| Sum | FILL IN |

2. bottle of wine purchase
| Contributor | Individual contribution amount ($) |
| ----------- | ---------------- |
| Sarah | FILL IN |
| ----------- | ---------------- |
| Sum | FILL IN |

3. gas purchase
| Contributor | Individual contribution amount ($) |
| ----------- | ---------------- |
| Jake | FILL IN |
| ----------- | ---------------- |
| Sum | FILL IN |

4. snacks purchase
| Contributor | Individual contribution amount ($) |
| ----------- | ---------------- |
| Jake | FILL IN |
| ----------- | ---------------- |
| Sum | FILL IN |

5. hotel room purchase
| Contributor | Individual contribution amount ($) |
| ----------- | ---------------- |
| Emily | FILL IN |
| ----------- | ---------------- |
| Sum | FILL IN |

6. dinner purchase
| Contributor | Individual contribution amount ($) |
| -----------

In [269]:
@tool
def add(first_float: float, second_float: float) -> float:
    """Adds two floats"""
    return first_float + second_float

@tool
def subtract(first_float: float, second_float: float) -> float:
    """Subtracts first_float from second_float"""
    return first_float - second_float

@tool
def divide(dividend: float, divisor: int) -> float:
    """Divides dividend by divisor."""
    return dividend / divisor

@tool
def multiply(float_factor: float, int_factor: int) -> float:
    """Multiplies float_factor by int_factor."""
    return float_factor * int_factor

@tool
def sum_floats(floats: List[float]) -> float:
    """Sums a list of floats"""
    return sum(floats)

tools = [add, subtract, divide, multiply, sum_floats]
llm_with_tools = llm.bind_tools(tools)

In [270]:
user_prompt = """Fill in the following Markdown tables given the passage below.

Only use math in the following situations:
1. To calculate individual contributions from a total, or
2. To calculate a total from individual contributions

Passage:
{input}

Markdown tables describing the purchases from the passage above and their contributors:
{tables}
"""

extract_contribution_amount_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know math",
        ),
        ("user", user_prompt),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [271]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "tables": lambda x: x["tables"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | extract_contribution_amount_prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [272]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [273]:
output_simple = list(agent_executor.stream({
    "input": inp_simple,
    "tables": "\n\n".join(prep_purchases(purchases_simple))
}))



> Entering new AgentExecutor chain...

Invoking: `add` with `{'first_float': 5, 'second_float': 8}`


13.0The total amount spent on the toilet plunger is $13.

Now, I will fill in the individual contribution amounts in the table:

| Contributor | Individual contribution amount ($) |
| ----------- | ---------------- |
| Joe | $5 |
| Hannah | $8 |
| ----------- | ---------------- |
| Sum | $13 |

> Finished chain.


In [274]:
output_complex = list(agent_executor.stream({
    "input": inp_complex,
    "tables": "\n\n".join(prep_purchases(purchases_complex))
}))



> Entering new AgentExecutor chain...

Invoking: `add` with `{'first_float': 60, 'second_float': 30}`


90.0
Invoking: `add` with `{'first_float': 50, 'second_float': 25}`


75.0
Invoking: `add` with `{'first_float': 150, 'second_float': 90}`


240.0
Invoking: `add` with `{'first_float': 80, 'second_float': 15}`


95.0
Invoking: `add` with `{'first_float': 20, 'second_float': 0}`


20.0
Invoking: `add` with `{'first_float': 20, 'second_float': 0}`


20.0The individual contributions for each purchase are as follows:

1. groceries purchase
| Contributor | Individual contribution amount ($) |
| ----------- | ---------------- |
| Sarah | $45.00 |
| ----------- | ---------------- |
| Sum | $45.00 |

2. bottle of wine purchase
| Contributor | Individual contribution amount ($) |
| ----------- | ---------------- |
| Sarah | $15.00 |
| ----------- | ---------------- |
| Sum | $15.00 |

3. gas purchase
| Contributor | Individual contribution amount ($) |
| ----------- | ---------------- |
| J

In [285]:
output_complex_str = output_complex[-1]['output']

In [ ]:
class 
class PurchaseContributor(BaseModel):
    """Information about a purchase"""
    name: str = Field(description="Descriptive name of the purchase")
    contributor_names: List[str] = Field(description="List of the names of people who contributed towards the purchase")
    consumer_names: List[str] = Field(description="List of the names of people who used/consumed the purchase")

extract_purchase_contributors_str = ChatPromptTemplate.from_template(
"""
Extract the desired information from the following Markdown tables.

Only extract the properties mentioned in the 'Purchases' function.

Derive the consumer of purchases by context. If not specified, list {people}.

Passage:
{input}
"""
)

extract_purchase_info_chain = extract_purchase_info_prompt | llm.with_structured_output(Purchases)

In [302]:
prompt = hub.pull("hwchase17/react")



In [304]:
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
